<a href="https://colab.research.google.com/github/kevinMGII/Grupo7-Practica1/blob/main/Grupo7_Practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PRIMERA PRÁCTICA**. Predicción del Abandono de Empleados.

Grupo 7. Miembros:

*   *Kevin Medina García, 100495893.*
*   *Bárbara Sánchez Moratalla, 100495857.*






---



### **CARGA Y VISUALIZACIÓN DE LOS DATOS**

En el desarrollo de esta práctica se utilizarán los archivos que contienen los datos con la extension **12**. Cumpliendo así las consideraciones generales, puesto que la suma de los últimos dos dígitos de nuestros NIA's es 12 en ambos casos.

A continuación se lee el conjunto de datos y se muestran las primeras filas.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

data_train = pd.read_csv("./attrition_availabledata_12.csv.gz")                 # Cargamos el conjunto de datos de entrenamiento
data_test = pd.read_csv("./attrition_competition_12.csv.gz")                    # Cargamos el conjunto de datos de test utilizando el almacenamiento local de Google Colab.

data_train.head()                                                               # Mostramos las 5 primeras filas del conjunto de datos de entrenamiento. (Not Working)
data_test.head()                                                                # Mostramos las 5 primeras filas del conjunto de datos de test.

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,6.894187,6,2,3,2.0,4.0,2.0,34,Travel_Rarely,Sales,6,4,Marketing,1,500,Male,2,Research Director,Divorced,96130,0.0,Y,12,8,1,16.0,5,15,0,9
1,7.684525,7,2,3,3.0,4.0,1.0,30,Non-Travel,Research & Development,15,3,Medical,1,3102,Female,2,Sales Executive,Married,23800,4.0,Y,18,8,3,9.0,2,5,1,4
2,6.806975,9,3,3,2.0,4.0,2.0,29,Travel_Frequently,Research & Development,16,4,Medical,1,1495,Male,1,Laboratory Technician,Single,65130,1.0,Y,11,8,1,10.0,2,10,0,9
3,6.028659,6,3,3,2.0,1.0,3.0,44,Travel_Rarely,Research & Development,18,4,Life Sciences,1,3472,Female,2,Sales Executive,Married,21190,9.0,Y,15,8,0,24.0,3,20,3,6
4,7.618041,12,2,3,3.0,4.0,3.0,28,Non-Travel,Research & Development,1,3,Medical,1,3627,Male,5,Sales Executive,Married,56790,0.0,Y,13,8,0,10.0,2,9,7,5


 ---

### **2. EDA SIMPLIFICADO**

A continuación, se realiza un Análisis Exploratorio de Datos (EDA). El objetivo es obtener una visión general del estado de los datos, permitiendo llevar a cabo algunos ajustes o limpiezas antes de empezar a utilizarlos.



**Número de  variables e instancias:**

In [2]:
num_instancias, num_variables = data_train.shape                                # Usamos el atributo .shape de pandas para obtener la tupla (número de filas, número de columnas)
print(f"Número de instancias: {num_instancias}")                                # Imprimimos num_instancias (número de filas)
print(f"Número de variables: {num_variables}")                                  # Imprimimos num_variables (número de columnas)

Número de instancias: 2940
Número de variables: 31


El conjunto de datos con el que vamos a llevar a cabo esta práctica esta formado por 2940 instancias y 31 variables. Disponemos de una cantidad suficiente de datos como para entrenar un modelo de forma correcta.

**Tipos de variables (categóricas, numéricas u ordinales):**

In [3]:
print(data_train.dtypes)                                                              # Mostramos los tipos de datos de cada columna del conjunto de datos de entrenamiento.

                                                                                      # Seleccionamos los tipos de datos categóricos y numéricos con pandas
categorical_vars = data_train.select_dtypes(include=['object', 'category']).columns   # Variables categóricas: aquellas de tipo 'object' o 'category'
numeric_vars = data_train.select_dtypes(include=['int64', 'float64']).columns         # Variables numéricas: aquellas de tipo 'int64' o 'float64'

print("Variables categóricas:", categorical_vars)                                     # Imprimimos las variables categóricas
print("Variables numéricas:", numeric_vars)                                           # Imprimimos las variables numéricas
# Las variables ordinales deben identificarse manualmente (mirar Nota)

hrs                        float64
absences                     int64
JobInvolvement               int64
PerformanceRating            int64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeID                   int64
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
Over18                      object
PercentSalaryHike            int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromot

Encontramos mayoritariamente variables numéricas en el conjunto de datos (23 variables numéricas frente a 8 variables categóricas).

*Nota: pandas no es capaz de identificar variables ordinales automáticamente.*


*Debemos analizar cada variable categórica y estudiar si existe un orden jerárquico entre los valores.*


```
# Ver valores únicos de la columna 'X' en el conjunto de entrenamiento
print(data_train['X'].unique())
```


* BusinessTravel (Non-Travel, Travel_Rarely, Travel_Frequently) -> No
* Department: (Sales, Research & Development, ...) -> No
* EducationField (Marketing, Medical, Life Sciences, ...) -> No
* Gender (Male, Female) -> No
* JobRole (Research Director, Sales Executive, Laboratory Technician, ...) -> No
* MaritalStatus (Single, Married, Divorced) -> No
* Over18 (N, Y) -> No
* Attrition (No, Yes) -> No

*Concluimos que no hay variables ordinales.*

**Variables categóricas con alta cardinalidad:**

In [4]:
for col in data_train.columns:                                                  # Recorremos las columnas del conjunto de datos: data_train
    tipo_columna = data_train[col].dtype                                        # Almacenamos en una variable el tipo de dato de la columna actual
    if tipo_columna == 'object' or tipo_columna.name == 'category':             # Check: ¿El tipo es un objeto, texto?, ¿El tipo es una categoría?
        valores_unicos = data_train[col].unique()                               # Almacenamos en una variable los valores únicos de esa categoría
        num_valores_unicos = len(valores_unicos)                                # Obtenemos la cantidad de valores únicos que hay
        print("Cardinalidad de", col, ":", num_valores_unicos)                  # Mostramos la cardinalidad y el número de valores únicos por pantalla

Cardinalidad de BusinessTravel : 3
Cardinalidad de Department : 3
Cardinalidad de EducationField : 6
Cardinalidad de Gender : 2
Cardinalidad de JobRole : 9
Cardinalidad de MaritalStatus : 3
Cardinalidad de Over18 : 1
Cardinalidad de Attrition : 2


El resultado obtenido nos indica que la mayoría de variables categóricas en el conjunto de datos presentan una baja cardinalidad. Sin embargo, algunas como "JobRole" tienen una alta cardinalidad con hasta 9 valores distintos.

Además, se puede observar como "Over18" solamente tiene un valor, por lo que no aporta información relevante y debe ser eliminada.

**Variables con valores faltantes y cuántos son:**

In [5]:
for col in data_train.columns:                                                  # Recorremos las columnas del conjunto de datos: data_train
    contador_faltantes = 0                                                      # Inicializamos el contador de faltantes a 0
    for valor in data_train[col]:                                               # Recorremos los valores de la columna
        if valor is None or valor != valor:                                     # Check: ¿El valor es nulo (faltante)? Especial: valor != valor para detectar NaN
            contador_faltantes += 1                                             # Incrementamos el contador de faltantes en una unidad
    if contador_faltantes > 0:                                                  # Check: ¿Hay valores faltantes en esa columna?
        print("Variable:", col, "- Valores faltantes:", contador_faltantes)     # Mostramos la variable con sus faltantes por pantalla

Variable: EnvironmentSatisfaction - Valores faltantes: 17
Variable: JobSatisfaction - Valores faltantes: 8
Variable: WorkLifeBalance - Valores faltantes: 23
Variable: NumCompaniesWorked - Valores faltantes: 12
Variable: TotalWorkingYears - Valores faltantes: 7


Tras la ejecución, se puede observar como hay cinco variables presentes en el modelo que contienen valores faltantes, destacando alguna como "WorkLifeBalance" con hasta 23 valores no presentes, o "EnvironmentSatisfaction" con hasta 17.

Durante el desarrollo de la práctica determinaremos qué estrategia seguir para solventar este problema, analizando el impacto de estas variables.

**Columnas constantes o columnas de ID:**

In [6]:
for col in data_train.columns:                                                  # Recorremos las columnas del conjunto de datos: data_train
    valores_unicos = data_train[col].unique()                                   # Obtenemos los valores únicos de la columna
    if len(valores_unicos) == 1 and valores_unicos[0] == valores_unicos[0]:     # Check: ¿La columna tiene un solo valor? Especial: ...[0] == ...[0] para detectar NaN
        print("Variable:", col, "- Columna constante")                          # Mostramos la variable con la columna constante
    elif len(valores_unicos) == len(data_train):                                # Check: ¿El número de valores únicos es igual al número de filas?
        print("Posible columna de ID:", col)                                    # Mostramos la posible variable o columna de ID por pantalla

Variable: EmployeeCount - Columna constante
Posible columna de ID: EmployeeID
Variable: Over18 - Columna constante
Variable: StandardHours - Columna constante


Se identifican tres columnas constantes: "EmployeeCount", "Over18" y "StandardHours", así como una posible columna de id: "EmployeeID".

Las columnas constantes no aportan información relevante o que resulte útil al modelo, y pueden eliminarse.

La columna de id contiene valores únicos para cada fila, por lo que no contribuye al modelo y puede eliminarse.

**Comprobación del tipo de problema: ¿clasificación o regresión?**

In [7]:
datos_col = data_train["Attrition"]                                             # Seleccionamos la columna "Attrition" del conjunto de datos de entrenamiento
valores_unicos = datos_col.unique()                                             # Obtenemos los valores únicos de la columna "Attrition"
print(valores_unicos)                                                           # Mostramos los valores únicos por pantalla

['No' 'Yes']


Como se puede observar en el resultado de la ejecución, se trata de un problema de clasificación binaria. El modelo podrá predecir una de las dos categorías: No, Yes.

**Puesto que es un problema de clasificación, ¿está desbalanceado?**

In [8]:
datos_col = data_train["Attrition"]                                             # Seleccionamos la columna "Attrition" del conjunto de datos de entrenamiento
contador_clases = datos_col.value_counts()                                      # Contamos la cantidad de valores de cada clase en la columna "Attrition"
print(contador_clases)                                                          # Mostramos la cantidad de valores de cada clase por pantalla

Attrition
No     2466
Yes     474
Name: count, dtype: int64


Concluimos el Análisis Exploratorio de Datos (EDA) indicando que el problema de clasificación está desbalanceado. Esto es debido a que existe una mayor cantidad de valores en una categoría: "No", respecto a la otra: "Yes".

Un conjunto de datos desbalanceado puede provocar que el modelo aprenda mejor la categoría con mayor cantidad de valores y no aprenda correctamente la categoría con menor cantidad de valores.



---



### **3. CÓMO SE VA A REALIZAR LA EVALUCIÓN**

Tras realizar el EDA simplicado y observar que se trata de un problema de clasificación binaria desbalanceado, definimos en esta sección la estrategia de evaluación con la que entrenar y evaluar el modelo.

Para comprender los resultados emplearemos diversas métricas:
* Accuracy: fracción de predicciones correctas (tanto positivas como negativas) sobre el total de predicciones.
* TPR (True Positive Rate): fracción de positivos reales correctamente identificados.
* TNR (True Negative Rate): fracción de negativos reales correctamente identificados.
* Balanced Accuracy: media del TPR y TNR, útil para datasets desbalanceados.
* Matriz de confusión: matriz que muestra la proporción de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

**3.1. División en Train y Test**

Antes de separar los datos en entrenamiento y test, limpiamos el conjunto de datos quitando aquellas variables que resultan irrelevantes, como se ha observado en el apartado segundo *(EmployeeID, EmployeeCount, Over18, StandardHours)*.

También eliminamos la columna 'Attrition' de las features X, ya que 'Attrition' es la variable a predecir y de lo contrario se produciría data leakage, pues el modelo entrenara con los datos de evaluación.

Por otro lado, debemos transformar las variables categóricas para que adopten valores numéricos. Para los casos en los que no hay relación entre los valores, se aplicará one-hot enconding; mientras que para los casos en los que sí, se mapearán de forma ordenada.

Similarmente, debemos transformar la columna *Attrition* de los datos de test. Al tratarse de una variable binaria, conviene transformar sus valores categóricos ("No" y "Yes") en numéricos (0 y 1 respectivamente). Es decir, aplicamos label encoding.

In [9]:
from sklearn.model_selection import train_test_split

X = data_train.drop(columns=["EmployeeID", "EmployeeCount", "Over18", "StandardHours", "Attrition"]) # Eliminamos para el conjunto train las columnas irrelevantes (y la columna de resultado 'Attrition')

X = pd.get_dummies(X, columns=['Department', 'EducationField', 'JobRole'], drop_first=True)          # Transformamos variables categóricas a numéricas sin orden con one-hot encoding (creamos columnas extras para cada valor dentro de la columna)

X.replace({                                                                                          # Transformamos variables categóricas a numéricas con orden
    'BusinessTravel': {'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2},
    'Gender': {'Male': 0, 'Female': 1},
    'MaritalStatus': {'Married': 0, 'Divorced': 1, 'Single': 2}
}, inplace=True)

y = data_train['Attrition'].map({'No': 0, 'Yes': 1})                                                 # Convertimos a valores binarios

<ipython-input-9-63b657c017f5>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X.replace({                                                                                          # Transformamos variables categóricas a numéricas con orden


Por último, si X_train o y_train tienen valores NaN, el modelo puede quedarse atascado. En nuestro caso tenemos originalmente 47 NaN en train, los cuales rellenamos con la media.

**3.2. Evaluación Outer: Estimación del rendimiento futuro**

La evaluación final del modelo se realizará con el método Holdout, por lo que dividiremos los datos en train (66.6%) y test (33.3%). Usaremos como semilla el NIA de uno de los miembros del grupo, como se exige en el enunciado.

Además, al tratarse de un problema desbalanceado, es importante estratificar los resultados *(stratify=y)* a la hora de hacer la separación para tener para tener el mismo desbalance en el conjunto de entrenamiento y en el de prueba. De lo contrario, el conjunto de entrenamiento podría no reflejar la distribución real de 'Attrition' y resultar poco útil para entrenar el modelo.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=100495893) # Dividimos en train y test
print("Número de instancias de train:", X_train.shape[0], "instancias")                                       # Imprimimos el número de instancias de entrenamiento
print("Número de instancias de test:", X_test.shape[0], "instancias")                                         # Imprimimos el número de instancias de test

Número de instancias de train: 1969 instancias
Número de instancias de test: 971 instancias


In [11]:
print(X_train.isnull().sum().sum())  # Número total de NaN
print(y_train.isnull().sum().sum())
X_train.fillna(X_train.mean(), inplace=True)  # Rellena NaN con la media

47
0


**3.3. Evaluación Inner: Optimización de Hiperparámetros y Comparación de Modelos**

La evaluación interna del modelo nos será útil para optimizar los hiperparámetros y elegir el mejor modelo.

Para realizarla aplicaremos validación cruzada en el conjunto de datos de entrenamiento y, puesto que se trata de un problema desbalanceado, dicha validación será estratificada (cada partición conservará la misma proporción de clases que el conjunto original).

In [12]:
from sklearn.model_selection import StratifiedKFold

cv_inner = StratifiedKFold(                                                     # Inner evaluation -> validación cruzada estratificada (StratifiedKFold)
    n_splits=5,                                                                 # División del conjunto de entrenamiento en 5 particiones o folds.
    shuffle=True,                                                               # Se mezclan los datos antes de dividirlos para mayor aleatoriedad
    random_state=100495893                                                      # Se fija una semilla para garantizar reproducibilidad, en este caso, el NIA.
)                                                                               # cv_inner es un objeto que utilizamos en el siguiente apartado

---

### **4. MÉTODOS BÁSICOS: KNN Y TREES**

En este apartado se llevará a cabo la evaluación de varios métodos de escalado e imputación utilizando K-Nearest Neighbors (KNN) como referencia. Mediante validación cruzada, se seleccionará la mejor combinación que se aplicará posteriormente.

- **Escalado e imputación usando KNN:**



In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

inner_scores = {}                                                               # Diccionario para almacenar los resultados de cada combinación

# Alternativa 1.1: KNN con StandardScaler ∧ Media
pipeline_std_mean = Pipeline([                                                  # Creacion del Pipeline: Imputer: mean | Scaler: standard | KNN
    ('imputer', SimpleImputer(strategy="mean")),                                # Imputación de valores faltantes con la media
    ('scaler', StandardScaler()),                                               # Escalado mediante StandardScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN)
])
scores_std = cross_val_score(pipeline_std_mean, X, y, cv=cv_inner, scoring="balanced_accuracy")             # Validación cruzada: usando cv_inner y balanced_accuracy (Apartado 3)
inner_scores['KNN con StandardScaler ∧ Media'] = scores_std.mean()                                                  # Se almacena el resultado en el diccionario inner_scores{}

# Alternativa 1.2: KNN con StandardScaler ∧ Mediana
pipeline_std_median = Pipeline([                                                # Creacion del Pipeline: Imputer: mean | Scaler: standard | KNN
    ('imputer', SimpleImputer(strategy="median")),                              # Imputación de valores faltantes con la media
    ('scaler', StandardScaler()),                                               # Escalado mediante StandardScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN)
])
scores_std = cross_val_score(pipeline_std_median, X, y, cv=cv_inner, scoring="balanced_accuracy")          # Validación cruzada: usando cv_inner y balanced_accuracy (Apartado 3)
inner_scores['KNN con StandardScaler ∧ Mediana'] = scores_std.mean()                                               # Se almacena el resultado en el diccionario inner_scores{}

# Alternativa 2.1: KNN con MinMaxScaler ∧ Media
pipeline_minmax_mean = Pipeline([                                                    # Creacion del Pipeline: Imputer: mean | Scaler: minmax | KNN
    ('imputer', SimpleImputer(strategy="mean")),                                # Imputación de valores faltantes con la media
    ('scaler', MinMaxScaler()),                                                 # Escalado mediante MinMaxScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN)
])
scores_minmax = cross_val_score(pipeline_minmax_mean, X, y, cv=cv_inner, scoring="balanced_accuracy")    # Validación cruzada: usando cv_inner y balanced_accuracy (Apartado 3)
inner_scores['KNN con MinMaxScaler ∧ Media'] = scores_minmax.mean()                                              # Se almacena el resultado en el diccionario inner_scores{}

# Alternativa 2.2: KNN con MinMaxScaler ∧ Mediana
pipeline_minmax_median = Pipeline([                                                    # Creacion del Pipeline: Imputer: mean | Scaler: minmax | KNN
    ('imputer', SimpleImputer(strategy="median")),                              # Imputación de valores faltantes con la media
    ('scaler', MinMaxScaler()),                                                 # Escalado mediante MinMaxScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN)
])
scores_minmax = cross_val_score(pipeline_minmax_median, X, y, cv=cv_inner, scoring="balanced_accuracy")    # Validación cruzada: usando cv_inner y balanced_accuracy (Apartado 3)
inner_scores['KNN con MinMaxScaler ∧ Mediana'] = scores_minmax.mean()                                              # Se almacena el resultado en el diccionario inner_scores{}

# Alternativa 3.1: KNN con RobustScaler ∧ Media
pipeline_robust_mean = Pipeline([                                               # Creacion del Pipeline: Imputer: mean | Scaler: robustscaler | KNN
    ('imputer', SimpleImputer(strategy="mean")),                                # Imputación de valores faltantes con la media
    ('scaler', RobustScaler()),                                                 # Escalado mediante RobustScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN)
])
scores_robust = cross_val_score(pipeline_robust_mean, X, y, cv=cv_inner, scoring="balanced_accuracy")    # Validación cruzada: usando cv_inner y balanced_accuracy (Apartado 3)
inner_scores['KNN con RobustScaler ∧ Media'] = scores_robust.mean()                                              # Se almacena el resultado en el diccionario inner_scores{}

# Alternativa 3.2: KNN con RobustScaler ∧ Mediana
pipeline_robust_median = Pipeline([                                             # Creacion del Pipeline: Imputer: mean | Scaler: robustscaler | KNN
    ('imputer', SimpleImputer(strategy="median")),                              # Imputación de valores faltantes con la media
    ('scaler', RobustScaler()),                                                 # Escalado mediante RobustScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN)
])
scores_robust = cross_val_score(pipeline_robust_median, X, y, cv=cv_inner, scoring="balanced_accuracy")    # Validación cruzada: usando cv_inner y balanced_accuracy (Apartado 3)
inner_scores['KNN con RobustScaler ∧ Mediana'] = scores_robust.mean()                                              # Se almacena el resultado en el diccionario inner_scores{}

# Mostrar los resultados
for name, score in inner_scores.items():                                        # Se recorren las claves del diccionario
    print(f"{name}: {score}")                                                   # Se imprime el nombre y el resultado

mejor_metodo = ""                                                               # Variable para almacenar el mejor método
mejor_puntuacion = 0                                                            # Variable para almacenar la mejor puntuación

for metodo, puntuacion in inner_scores.items():                                 # Se recorren los pares clave-valor del diccionario
    if puntuacion > mejor_puntuacion:                                           # Check: la puntuación actual es mejor que la mejor puntuación almacenada
        mejor_puntuacion = puntuacion                                           # Se actualiza la mejor puntuación
        mejor_metodo = metodo                                                   # Se actualiza el mejor método

print("\nEl mejor método seleccionado es:", mejor_metodo)                         # Mostramos por pantalla el mejor método seleccionado

KNN con StandardScaler ∧ Media: 0.6230156483123099
KNN con StandardScaler ∧ Mediana: 0.6228128085557177
KNN con MinMaxScaler ∧ Media: 0.6174301570946651
KNN con MinMaxScaler ∧ Mediana: 0.6178358366078497
KNN con RobustScaler ∧ Media: 0.6155035830247813
KNN con RobustScaler ∧ Mediana: 0.615300743268189

El mejor método seleccionado es: KNN con StandardScaler ∧ Media


**KNN y árboles**

In [14]:
import time
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

- KNN con hiperparámetros por omisión

In [15]:
pipeline_knn = Pipeline([                                                       # Creacion del Pipeline: Imputer: mean | Scaler: standard | KNN
    ('imputer', SimpleImputer(strategy="mean")),                                # Imputación de valores faltantes con la media
    ('scaler', StandardScaler()),                                               # Escalado mediante StandardScaler
    ('knn', KNeighborsClassifier())                                             # Clasificador K-Nearest Neighbors (KNN). Por defecto, n_neigbors = 5
    ])

inicio = time.time()                                                                           # Tiempo de inicio
scores_KNN = cross_val_score(pipeline_knn, X, y, cv=cv_inner, scoring="balanced_accuracy")     # Validación cruzada: usando cv_inner y balanced_accuracy
print(scores_KNN)                                                                              # Muestra los resultados de la validación cruzada
tiempo_KNN = time.time() - inicio                                                              # Tiempo de ejecución
score_KNN = np.mean(scores_KNN)                                                                # Media de los resultados de la validación cruzada

print("Score=", score_KNN, ", Tiempo=", tiempo_KNN, "s")                                       # Muestra el score y el tiempo de ejecución

[0.63831941 0.61877869 0.66272019 0.61757233 0.57768763]
Score= 0.6230156483123099 , Tiempo= 0.29850053787231445 s


- Árbol de decisión con parámetros por omisión

Decision Tree Classifier

In [16]:
from sklearn.tree import DecisionTreeClassifier

pipeline_dt = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),                                # Manejo de valores faltantes con la media
    ('scaler', StandardScaler()),                                               # Estandarización de datos
    ('dt', DecisionTreeClassifier())                                            # Árbol de decisión con pesos balanceados
])

inicio = time.time()                                                                      # Tiempo de inicio
scores_DT = cross_val_score(pipeline_dt, X, y, cv=cv_inner, scoring="balanced_accuracy")  # Validación cruzada: usando cv_inner y balanced_accuracy
print(scores_DT)                                                                          # Muestra los resultados de la validación cruzada
tiempo_DT = time.time() - inicio                                                          # Tiempo de ejecución
score_DT = np.mean(scores_DT)                                                             # Media de los resultados de la validación cruzada

print("Score=", score_DT, ", Tiempo=", tiempo_DT, "s")                          # Muestra el score y el tiempo de ejecución

[0.87591524 0.84408028 0.84325825 0.84630084 0.86329668]
Score= 0.8545702580550133 , Tiempo= 0.24579739570617676 s


Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

pipeline_rf = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),                                # Manejo de valores faltantes con la media
    ('scaler', StandardScaler()),                                               # Estandarización de datos
    ('rf', RandomForestClassifier())                                            # Árbol de decisión con pesos balanceados
])

inicio = time.time()                                                                        # Tiempo de inicio
scores_RF = cross_val_score(pipeline_rf, X, y, cv=cv_inner, scoring="balanced_accuracy")    # Validación cruzada: usando cv_inner y balanced_accuracy
print(scores_RF)                                                                            # Muestra los resultados de la validación cruzada
tiempo_RF = time.time() - inicio                                                            # Tiempo de ejecución
score_RF = np.mean(scores_RF)                                                               # Media de los resultados de la validación cruzada

print("Score=", score_RF, ", Tiempo=", tiempo_RF, "s")                          # Muestra el score y el tiempo de ejecución

[0.87690585 0.86842105 0.89270844 0.85688054 0.89372264]
Score= 0.8777277054925386 , Tiempo= 2.5452730655670166 s


Balanced Random Forest

In [18]:
from imblearn.ensemble import BalancedRandomForestClassifier

pipeline_brf = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),                                # Manejo de valores faltantes con la media
    ('scaler', StandardScaler()),                                               # Estandarización de datos
    ('brf', BalancedRandomForestClassifier())                                   # Árbol de decisión con pesos balanceados
])

inicio = time.time()                                                                        # Tiempo de inicio
scores_BRF = cross_val_score(pipeline_brf, X, y, cv=cv_inner, scoring="balanced_accuracy")  # Validación cruzada: usando cv_inner y balanced_accuracy
print(scores_BRF)                                                                           # Muestra los resultados de la validación cruzada
tiempo_BRF = time.time() - inicio                                                           # Tiempo de ejecución
score_BRF = np.mean(scores_BRF)                                                             # Media de los resultados de la validación cruzada

print("Score=", score_BRF, ", Tiempo=", tiempo_BRF, "s")                        # Muestra el score y el tiempo de ejecución

[0.92607029 0.88517135 0.94065336 0.90989644 0.92708445]
Score= 0.9177751768797971 , Tiempo= 2.3519439697265625 s


- KNN con ajuste de hiperparámetros

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid_knn = {
    'knn__n_neighbors': [3, 5, 7, 9],                                           # Número de vecinos a considerar
    'knn__weights': ['uniform', 'distance'],                                    # Ponderación de los vecinos
    'knn__metric': ['euclidean', 'manhattan']                                   # Métrica de distancia
}
                                                                                # Encuentra la mejor combinación de hiperparámetros
grid_search_knn = GridSearchCV(pipeline_knn,                                    # Pipeline del modelo KNN
                               param_grid_knn,                                  # Hiperparámetros a ajustar
                               cv=cv_inner,                                     # Evaluación interna (inner)
                               scoring="balanced_accuracy",                     # Métrica para la evaluación
                               n_jobs=-1)                                       # Usar todos los núcleos disponibles

inicio = time.time()                                                            # Obtenemos el tiempo inicial
grid_search_knn.fit(X, y)                                                       # Busca los mejores hipérparametros y entrena el modelo
tiempo_gs_brf = time.time() - inicio                                            # Calculamos el tiempo total

best_knn = grid_search_knn.best_estimator_                                      # KNN con la mejor config
best_score_knn = grid_search_knn.best_score_                                    # Mejor puntuación de la validación cruzada
print("Mejor configuración KNN:", grid_search_knn.best_params_)                 # Muestra por pantalla los mejores hiperparámetros
print("Mejor Score KNN:", best_score_knn, ", Tiempo=", tiempo_gs_brf, "s")      # Muestra por pantalla la puntuación

Mejor configuración KNN: {'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}
Mejor Score KNN: 0.9142432420673824


- Árbol de decisión con ajuste de hiperparámetros

Decision Tree Classifier

In [20]:
param_grid_dt = {
    'dt__max_depth': [5, 10, 15, None],                                         # Profundidad máxima del árbol
                                                                                # No me gusta mucho None, permite que el árbol se expanda sin limite.
    'dt__min_samples_split': [2, 5, 10],                                        # Número mínimo de muestras necesarias para dividir un nodo
    'dt__min_samples_leaf': [1, 2, 5]                                           # Número mínimo de muestras necesarias en una hoja
}

grid_search_dt = GridSearchCV(pipeline_dt,                                      # Pipeline del modelo DT
                              param_grid_dt,                                    # Hiperparámetros a ajustar
                              cv=cv_inner,                                      # Evaluación interna (inner)
                              scoring="balanced_accuracy",                      # Métrica para la evaluación
                              n_jobs=-1)                                        # Usar todos los núcleos disponibles

inicio = time.time()                                                            # Obtenemos el tiempo inicial
grid_search_dt.fit(X, y)                                                        # Busca los mejores hiperparámetros y entrena el modelo
tiempo_gs_brf = time.time() - inicio                                            # Calculamos el tiempo total

best_dt = grid_search_dt.best_estimator_                                        # Decision Tree con la mejor config
best_score_dt = grid_search_dt.best_score_                                      # Mejor puntuación de la validación cruzada
print("Mejor configuración DT:", grid_search_dt.best_params_)                   # Muestra por pantalla los mejores hiperparámetros
print("Mejor Score DT:", best_score_dt, ", Tiempo=", tiempo_gs_brf, "s")        # Muestra por pantalla la puntuación

Mejor configuración DT: {'dt__max_depth': None, 'dt__min_samples_leaf': 1, 'dt__min_samples_split': 2}
Mejor Score DT: 0.8689856771613685 , Tiempo= 5.4299681186676025 s


Random Forest Classifier

In [21]:
param_grid_rf = {
    'rf__n_estimators': [50, 100, 200],                                         # Número de árboles en el bosque
    'rf__max_depth': [5, 10, None],                                             # Profundidad máxima de cada árbol
                                                                                # No me gusta mucho None, permite que el árbol se expanda sin limite.
    'rf__min_samples_split': [2, 5, 10],                                        # Número mínimo de muestras necesarias para dividir un nodo
    'rf__min_samples_leaf': [1, 2, 5]                                           # Número mínimo de muestras necesarias en una hoja
}

grid_search_rf = GridSearchCV(pipeline_rf,                                      # Pipeline del modelo RF
                              param_grid_rf,                                    # Hiperparámetros a ajustar
                              cv=cv_inner,                                      # Evaluación interna (inner)
                              scoring="balanced_accuracy",                      # Métrica para la evaluación
                              n_jobs=-1)                                        # Usar todos los núcleos disponibles

inicio = time.time()                                                            # Obtenemos el tiempo inicial
grid_search_rf.fit(X, y)                                                        # Busca los mejores hiperparámetros y entrena el modelo
tiempo_gs_brf = time.time() - inicio                                            # Calculamos el tiempo total

best_rf = grid_search_rf.best_estimator_                                        # Random Forest con la mejor config
best_score_rf = grid_search_rf.best_score_                                      # Mejor puntuación de la validación cruzada
print("Mejor configuración RF:", grid_search_rf.best_params_)                   # Muestra por pantalla los mejores hiperparámetros
print("Mejor Score RF:", best_score_rf, ", Tiempo=", tiempo_gs_brf, "s")        # Muestra por pantalla la puntuación

Mejor configuración RF: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
Mejor Score RF: 0.88552892008491 , Tiempo= 203.39189052581787 s


Balanced Random Forest

In [22]:
param_grid_brf = {
    'brf__n_estimators': [50, 100, 200],                                        # Número de árboles en el bosque
    'brf__max_depth': [10, 20, None],                                           # Profundidad máxima de cada árbol
                                                                                # No me gusta mucho None, permite que el árbol se expanda sin limite.
    'brf__min_samples_split': [2, 5, 10],                                       # Número mínimo de muestras necesarias para dividir un nodo
    'brf__min_samples_leaf': [1, 2, 4]                                          # Número mínimo de muestras necesarias en una hoja
}

gs_brf = GridSearchCV(pipeline_brf,                                             # Pipeline del modelo BRF
                      param_grid_brf,                                           # Hiperparámetros a ajustar
                      cv=cv_inner,                                              # Evaluación interna (inner)
                      scoring="balanced_accuracy",                              # Métrica para la evaluación
                      n_jobs=-1)                                                # Usar todos los núcleos disponibles

inicio = time.time()                                                            # Obtenemos el tiempo inicial
gs_brf.fit(X, y)                                                                # Busca los mejores hiperparámetros y entrena el modelo
tiempo_gs_brf = time.time() - inicio                                            # Calculamos el tiempo total

print("Mejor configuración BRF:", gs_brf.best_params_)                          # Muestra por pantalla los mejores hiperparámetros
print("Mejor score BRF:", gs_brf.best_score_, ", Tiempo=", tiempo_gs_brf, "s")  # Muestra por pantalla la puntuación

Mejor configuración BRF: {'brf__max_depth': 20, 'brf__min_samples_leaf': 1, 'brf__min_samples_split': 2, 'brf__n_estimators': 100}
Mejor score BRF: 0.9251547961407155 , Tiempo= 226.7458176612854 s
